# 1) Importing query and collection data

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
PATH_COLLECTION_DATA = 'subtask_4b/subtask4b_collection_data.pkl' 

df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [3]:
PATH_QUERY_TRAIN_DATA = 'subtask_4b/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA = 'subtask_4b/subtask4b_query_tweets_dev.tsv' 

df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [4]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [5]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


# 2) Running the BM25 baseline
The following code runs a BM25 baseline.


In [15]:
from rank_bm25 import BM25Okapi

In [16]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:100] # @k: how many docs shall the ranked list include?
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [10]:
# Retrieve top50 candidates using the BM25 model

train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

if not os.path.exists(train_pkl_path):
    df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_train.to_pickle(train_pkl_path)
else:
    df_query_train = pd.read_pickle(train_pkl_path)

if not os.path.exists(dev_pkl_path):
    df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_dev.to_pickle(dev_pkl_path)
else:
    df_query_dev = pd.read_pickle(dev_pkl_path)

# 3) Neural Re-Ranking

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm
import json
from pathlib import Path
import re

## 3.1) Baseline: ColBERT architecture with SciBERT
Use a pretrained SciBERT model to:
- embed each query-token
- embed each doc-token (can be pre-computed)

For each query-doc pair:
- calculate match-matrix: each query-token – doc-token pair gets cosine similarity value
- aggregate the score: 
    - for each query-token take max cosine similarity value with corresponding doc-tokens
    - sum over all of the max elements

In [7]:
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden_dim]
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    token_embeddings = token_embeddings[attention_mask] 
    return token_embeddings  # Shape: [num_tokens, hidden_dim]

def build_and_save_doc_embeddings(
    docs_df,
    model_name,
    save_dir,
    max_len=512,
    device="cpu"
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    save_path = Path("doc_embeddings_" + save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata_path = save_path / "metadata.json"
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        metadata = {}

    for i, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_id = row.get("cord_uid", f"doc_{i}")
        file_path = save_path / f"{doc_id}.pt"

        if file_path.exists() and doc_id in metadata:
            continue

        text = str(row.get('title', '')) + " " + str(row.get('abstract', '')) + " Authors: " + str(row.get('authors', ''))

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            output = model(**inputs)
            token_embeddings = output.last_hidden_state.squeeze(0)
            attention_mask = inputs['attention_mask'].squeeze(0).bool()
            token_embeddings = token_embeddings[attention_mask]

        n_tokens = token_embeddings.size(0)
        pad_len = max_len - n_tokens

        if pad_len > 0:
            padding = torch.zeros(pad_len, token_embeddings.size(1), device=device)
            token_embeddings = torch.cat([token_embeddings, padding], dim=0)
        else:
            token_embeddings = token_embeddings[:max_len]

        try:
            torch.save(token_embeddings.cpu(), file_path)
        except Exception as e:
            print(f"Fehler beim Speichern von {doc_id}: {e}")
            continue

        metadata[doc_id] = {
            "title": row.get("title", ""),
            "abstract": row.get("abstract", ""),
            "authors": row.get("authors", ""),
            "length": min(n_tokens, max_len),
            "path": str(file_path)
        }

    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    return metadata


In [14]:
def pre_compute_embeddings(save_name):
    if not os.path.exists("doc_embeddings_" + save_name + "/metadata.json"):
        metadata = build_and_save_doc_embeddings(df_collection, model_name=model_name, save_dir=save_name, device="cuda")
    else:
        with open("doc_embeddings_" + save_name + "/metadata.json", "r") as f:
            metadata = json.load(f)
    return metadata

In [9]:
def rerank(df, metadata, tokenizer, model, save_name):
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        tweet_text = row['tweet_text']
        pre_ranked_docs = row['bm25_topk']

        q_emb = get_token_embeddings(tweet_text, tokenizer, model)
        q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

        scores = []
        for doc in pre_ranked_docs:
            emb = torch.load(metadata[doc]["path"])
            length = metadata[doc]["length"]
            d_emb = emb[:length]
            d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

            sim_matrix = torch.matmul(q_norm, d_norm.T)
            max_sim_per_q = sim_matrix.max(dim=1).values
            score = max_sim_per_q.sum().item()
            scores.append(score)

        df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df


In [20]:
# settings for model run:
model_name = "allenai/scibert_scivocab_uncased"
save_name = "scibert_baseline"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [21]:
# pre-compute embeddings
metadata = pre_compute_embeddings(save_name)

100%|██████████| 7718/7718 [03:24<00:00, 37.67it/s]


In [ ]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, save_name)

  0%|          | 0/1400 [00:00<?, ?it/s]<ipython-input-15-a0c9f489743a>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(metadata[doc]["path"])
100%|██████

In [ ]:
# re-rank BM25 list for train data
df_query_train = rerank(df_query_train, metadata, tokenizer, model)

  0%|          | 0/12853 [00:00<?, ?it/s]<ipython-input-15-a0c9f489743a>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(metadata[doc]["path"])
100%|█████

## 3.2) ColBERT w/ fine-tuned SciBERT

In [10]:
train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

df_query_dev = pd.read_pickle(dev_pkl_path)
df_query_train = pd.read_pickle(train_pkl_path)

In [11]:
from torch.utils.data import Dataset, DataLoader
import random
import torch.nn.functional as F

In [16]:
model_name = "allenai/scibert_scivocab_uncased"
save_name = "colB_sciB_1"
metadata = pre_compute_embeddings(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [17]:
# hyperparameters
BATCH_SIZE = 8
EPOCHS = 4
LR = 2e-5
MARGIN = 0.2

class ColBERTTripletDataset(Dataset):
    def __init__(self, df, metadata, tokenizer, num_negatives=1):
        self.data = []
        self.tokenizer = tokenizer
        self.metadata = metadata
        for _, row in df.iterrows():
            query = row["tweet_text"]
            pos = row["cord_uid"]
            negatives = [doc for doc in row["bm25_topk"] if doc != pos]
            if negatives:
                for _ in range(num_negatives):
                    neg = random.choice(negatives)
                    self.data.append((query, pos, neg))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def colbert_score_from_emb(q_emb, d_emb):
    q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)
    d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)
    sim_matrix = torch.matmul(q_norm, d_norm.T)
    max_sim_per_q = sim_matrix.max(dim=1).values
    return max_sim_per_q.sum()

# create training triplets
train_dataset = ColBERTTripletDataset(df_query_train, metadata, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# optimizer
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.train()
model.to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        queries, pos_ids, neg_ids = batch

        inputs = tokenizer(list(queries), return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        outputs = model(**inputs)
        q_emb_batch = outputs.last_hidden_state  # [B, L, D]
        attention_mask = inputs["attention_mask"].bool()
        q_embs = [emb[mask] for emb, mask in zip(q_emb_batch, attention_mask)]  # list of [num_tokens, D]

        score_pos_list = []
        score_neg_list = []

        for i in range(len(queries)):
            d_pos_emb = torch.load(metadata[pos_ids[i]]["path"]).to(DEVICE)[:metadata[pos_ids[i]]["length"]]
            d_neg_emb = torch.load(metadata[neg_ids[i]]["path"]).to(DEVICE)[:metadata[neg_ids[i]]["length"]]

            q_emb = q_embs[i]
            score_pos = colbert_score_from_emb(q_emb, d_pos_emb)
            score_neg = colbert_score_from_emb(q_emb, d_neg_emb)

            score_pos_list.append(score_pos)
            score_neg_list.append(score_neg)

        score_pos_batch = torch.stack(score_pos_list)
        score_neg_batch = torch.stack(score_neg_list)

        loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

Epoch 1:   0%|          | 0/1607 [00:00<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [28]:
model.save_pretrained(save_name)
tokenizer.save_pretrained(save_name)

('colB_sciB_3/tokenizer_config.json',
 'colB_sciB_3/special_tokens_map.json',
 'colB_sciB_3/vocab.txt',
 'colB_sciB_3/added_tokens.json',
 'colB_sciB_3/tokenizer.json')

#### Test run:

In [29]:
# settings for model run:
model_name = save_name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [30]:
# pre-compute embeddings
metadata = pre_compute_embeddings(save_name)

100%|██████████| 7718/7718 [19:44<00:00,  6.51it/s]


In [31]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, save_name)

  0%|          | 0/1400 [00:00<?, ?it/s]<ipython-input-17-25a755989e47>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(metadata[doc]["path"])
100%|██████

In [ ]:
# re-rank BM25 list for train data
df_query_train = rerank(df_query_train, metadata, tokenizer, model, save_name)

## 3.3) ColBERT w/ fine-tuned SciBERT for docs and CTBERT for queries

# 4) Evaluation
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [24]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [23]:
# ---- BM25 Baseline ----
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print("---- BM25 Baseline ----")
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

---- BM25 Baseline ----
Results on the train set: {1: 0.5079747918773827, 5: 0.5508999196037242, 10: 0.5558827906275973}
Results on the dev set: {1: 0.505, 5: 0.5520357142857142, 10: 0.5574200680272109}


In [24]:
# ---- ColBERT Re-Ranking Baseline ----
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'scibert_baseline_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'scibert_baseline_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print("---- Re-Ranking Baseline: ColBERT (SciBERT) ----")
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

---- Re-Ranking Baseline: ColBERT (SciBERT) ----
Results on the train set: {1: 0.5081303975725512, 5: 0.5548354469773593, 10: 0.5600201422927634}
Results on the dev set: {1: 0.53, 5: 0.569404761904762, 10: 0.5742800453514738}


In [ ]:
# ---- ColBERT Re-Ranking Finetune ----
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'scibert_finetune-1_topk')
print("---- Re-Ranking Finetune-1: ColBERT (SciBERT) ----")
print(f"Results on the dev set: {results_dev}")

---- Re-Ranking Finetune-1: ColBERT (SciBERT) ----
Results on the dev set: {1: 0.5978571428571429, 5: 0.6351428571428571, 10: 0.6379787414965986}


In [25]:
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'colB_sciB_2_topk')
print("---- Re-Ranking Finetune-2: ColBERT (SciBERT) ----")
print(f"Results on the dev set: {results_dev}")

---- Re-Ranking Finetune-2: ColBERT (SciBERT) ----
Results on the dev set: {1: 0.6071428571428571, 5: 0.6401666666666667, 10: 0.6434824263038549}


In [32]:
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'colB_sciB_3_topk')
print("---- Re-Ranking Finetune-3: ColBERT (SciBERT) ----")
print(f"Results on the dev set: {results_dev}")

---- Re-Ranking Finetune-3: ColBERT (SciBERT) ----
Results on the dev set: {1: 0.5828571428571429, 5: 0.6225952380952381, 10: 0.6260402494331067}


## Results documentation

#### 1) SciBERT in ColBERT architecture
Re-Ranking of top 50 BM25 results for each query:
- SciBERT (ColBERT)
    - used SciBERT model out of the box
- SciBERT (ColBERT) @ 2EP
    - fine tuned SciBERT in 2 epochs on train data
- SciBERT (ColBERT) @ 4EP
    - based on previous fine tune (SciBERT (ColBERT) @ 2EP), fine tuned SciBERT in additional 2 epochs, resulting in 4 epochs fine tune compared to out of the box SciBERT
- SciBERT (ColBERT) @ 8EP
    - based on previous fine tune (SciBERT (ColBERT) @ 4EP), fine tuned SciBERT in additional 4 epochs, resulting in 8 epochs fine tune compared to out of the box SciBERT

Hyperparameters:
- BATCH_SIZE = 8
- LR = 2e-5
- MARGIN = 0.2

Loss Function: <br>
loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
<br>
<br>

|Model                        | MRR@5 (dev)     |
|-----------------------------|-----------------|
|BM25 (baseline)              |55.20%           |
|SciBERT (ColBERT)            |56.94%           | 
|SciBERT (ColBERT) @ 2EP      |63.51%           |
|SciBERT (ColBERT) @ 4EP      |64.02%           |
|SciBERT (ColBERT) @ 8EP      |62.26%           |


# 5) Exporting results to prepare the submission on Codalab

In [21]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [22]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')